In [ ]:
# ## Data model ##
# {
#     _id: <ObjectID>,
#     date: "28-02-2009", ## convert to mongodb date format before insertion
#     climate: {
#         station: "2002-022A",
#         air_temp: 23,
#         relative_humidity: 54.1,
#         wind_speed_knots: 5.7
#     },
#     fire: [
#         {
#             latitude: -37.355,
#             longitude: 146.06, 
#             time: "12:53:50",
#             power: 76.4,
#             confidence: 100,
#             surface_temp: 77  
#         },
#         {
#             latitude: -37.357,
#             longitude: 146.046,
#             datetime: "12:53:50",
#             power: 92.4,
#             confidence: 100,
#             surface_temp: 83  
#         },
#         {
#             latitude: -37.346,
#             longitude: 146.044,
#             datetime: "12:53:50",
#             power: 20,
#             confidence: 81,
#             surface_temp: 40  
#         }
#     ]

# }

"""JUSTIFICATION
I have tried to reduce data redundancy where possible with this model, hence I have gone with the 
data embedding model. In our data set there is a 1:N or 1:0 relationship between climate date and fire date,
so it made sense to embed the fire data inside the climate data where the relationship exists so as not to 
repeat fire data unnecessarily, while still maintaining the relationship with climate data.
This also reduces the number of searches on the data when looking for fire data by date. I also do not expect 
the size of each document to be greater than 16MB (the size limit) so an embedding model would be appropriate here
"""

In [ ]:
# Import required modules
import csv
import json
import sys
import pprint
import pymongo

from pprint import pprint
from datetime import datetime 

from pymongo import MongoClient
client = MongoClient()
db = client.fit5148_assignment_db
climate_fire = db.climate_fire

In [ ]:
climate_historic = []
fire_historic = []

# Open the climate_historic CSV file 
with open('climate_historic.csv', 'r', encoding='utf-8-sig') as climate_historic_file: 
    # Skip header line
    next(climate_historic_file)
    
  # Read the fire_historic CSV file, append each row to the array
    for row in csv.reader(climate_historic_file):
        climate_historic.append(row) 
        
        
# Open the fire_historic CSV file
with open('fire_historic.csv', 'r', encoding='utf-8-sig') as fire_historic_file: 
    # Skip header line
    next(fire_historic_file)
    
  # Read the fire_historic CSV file, append each row to the array
    for row in csv.reader(fire_historic_file):
        fire_historic.append(row) 

In [ ]:
# Date sorting function - the two files have different types of date data, so we need to handle both
def date_sort_function(data, column, file_type):

    input_data = list(data)
    if file_type == "fire":
        
        # Convert fire date string to datetime object to enable sorting on this value
        date_sorted = sorted(input_data, key=lambda x: datetime.strptime(x[column], '%Y-%m-%dT%H:%M:%S'))
    
    # Convert climate date string to datetime object to enable sorting on this value
    elif file_type == "climate":
        date_sorted = sorted(input_data, key=lambda x: datetime.strptime(x[column], '%Y-%m-%d'))

    return date_sorted


In [ ]:
def SM_join(s_T1, T1_index, s_T2, T2_index):
#  Input s_T1 is a sorted list, s_T2 is a sorted list

    result = []

    i = j = 0
    while True:
        r = s_T1[i][T1_index]
        r = r[:10]
            
        s = s_T2[j][T2_index]

        # If join attribute s_T1(i) < join attribute s_T2(i)
        if r < s:
            result.append([s_T2[j][1], s_T2[j][0], s_T2[j][2], s_T2[j][3], s_T2[j][4]])
            # Iterate to next entry in s_T1
            i = i + 1
            

        # If join attribute s_T1(1) > join attribute s_T2(1)
        elif r > s:
            result.append([s_T2[j][1], s_T2[j][0], s_T2[j][2], s_T2[j][3], s_T2[j][4]])
            # Iterate to next entry in s_T2
            j = j + 1


        # If values are a match
        elif r == s :

                # Append values that match to result[]
                result.append([s_T2[j][1], s_T2[j][0], s_T2[j][2], s_T2[j][3], s_T2[j][4], s_T1[i][0], s_T1[i][1], s_T1[i][2][11:], s_T1[i][3], s_T1[i][4], s_T1[i][5]])

                # iterate to next r value
                i = i + 1
                if i < len(s_T1):
                    r = s_T1[i][T1_index]
                    r = r[:10]

                    
                # Since there are repeated dates in table 1 (fire_partitions), we want all of these values. 
                # Only when the date in table 1 becomes greater than table 2, then we iterate 
                # to the next value in table 2.
                if r > s:
                    j = j + 1

        # if either s_T1(i) or s_T2(j) is EOF Then break
        if (i == len(s_T1)) or (j == len(s_T2)):
            break

    return result



In [ ]:
fire_datetime_sorted = date_sort_function(fire_historic, 2, "fire")
result1 = SM_join(fire_datetime_sorted, 2, climate_historic, 1)
# print(result1)

In [ ]:
data = {}
    
for row in result1:
    # create date key from the date value
	date_key = str(row[0]).replace('-', '')

    # If date key does not exist in dict, create it and add climate data
	if date_key not in data:
		data[date_key] = {
			"date": row[0],
			"climate": {
				"station": row[1],
				"air_temp": int(row[2]),
				"relative_humidity": float(row[3]),
				"wind_speed_knots": float(row[4]),
			}
		}

    # If the length of data is >5 and there is no existing fire data in the entry, 
    # create new fire list and add data
	if len(row) > 5 and 'fire' not in data[date_key]:
		data[date_key]['fire'] = [
			{
				"latitude": float(row[5]),
				"longitude": float(row[6]),
				"time": row[7],
				"power": float(row[8]),
				"confidence": int(row[9]),
				"surface_temp": float(row[10])
			}
		]

    # If the length of data is >5 and there is already existing fire entries in the fire list, append new entry
	elif len(row) > 5 and 'fire' in data[date_key]:
		data[date_key]['fire'].append(
			{
				"latitude": float(row[5]),
				"longitude": float(row[6]),
				"time": row[7],
				"power": float(row[8]),
				"confidence": int(row[9]),
				"surface_temp": float(row[10])
			}
		)

In [ ]:
data_list = []

# Add the data to the db
for key, val in data.items():
    data_list.append(val)


for dt in data_list:
    result = climate_fire.insert_one(dt)


In [ ]:
# (A) Find air temperature and relative humidity on 15th October 2019
res_a = list(climate_fire.find({"date":"2019-10-15"},
	{"date": 1,
		"climate.air_temp": 1,
		"climate.relative_humidity": 1
	})
)

# print(res1)
for i in res_a:
	air_temp = i["climate"]["air_temp"]
	rel_hum = i["climate"]["relative_humidity"]

print("On 15th October 2019:\nAir Temperature:",str(air_temp)+u"\N{DEGREE SIGN}C\nRelative Humidity:", str(rel_hum)+"%")

In [ ]:
# (B) Find the latitude, longitude, surface temperature and confidence when the
# surface temperature (°C) was between 65 °C and 100 °C . The BETWEEN
# condition needs to return the records where expression is within the range
# of 65 °C and 100 °C (inclusive).

res_b = climate_fire.aggregate([
	{
		"$project": {
			"fire": {
				"$filter": {
					"input": "$fire",
					"as": "fire",
					"cond": { "$and": [
								{"$gte": ["$$fire.surface_temp", 65]},
								{"$lte": ["$$fire.surface_temp", 100]}
							]
					}
				}
			}
		}
	}
])

for i in list(res_b):
	if i["fire"]:
		for each in i["fire"]:
			print("+-------------------------------------------------------------------------------------+")
			print("Latitude:",each["latitude"], "| Longitude:", each["longitude"], "| Surface Temperature:", str(each["surface_temp"])+u"\N{DEGREE SIGN}C", "| Confidence", each["confidence"])

In [ ]:
# (C) Find date, surface temperature (°C), air temperature (°C), relative humidity and
# wind speed (wind_speed_knots) on 2nd and 3rd of October 2019.

res_c = list(climate_fire.find(
	{"$or": [
		{"date":"2019-10-02"},
		{"date":"2019-10-03"},
	]},
	{
		"_id": 0,
		"date": 1,
		"fire.surface_temp": 1,
		"climate.air_temp": 1,
		"climate.relative_humidity": 1,
		"climate.wind_speed_knots":1
	}
))
pprint(res_c)

In [ ]:
# (D) Find datetime, air temperature (°C), surface temperature (°C) and confidence
# when the confidence is between 90 and 100. The BETWEEN condition
# needs to return the records where expression is within the range of 90 and
# 100 (inclusive).

res_d = list(climate_fire.aggregate([
	{
		"$project": {
			"fire": {
				"$filter": {
					"input": "$fire",
					"as": "fire",
					"cond": {"$and": [
								{"$gte": ["$$fire.confidence", 90]},
								{"$lte": ["$$fire.confidence", 100]}
							]}
				}
			},
			"climate.air_temp": 1,
			"date": 1,
			"_id": 0
		}
	}
]))

for i in res_d:
	if i["fire"]:
		for each in i["fire"]:
			print("+-------------------------------------------------------------------------------------------------------------+")
			print("Datetime:",str(i["date"])+"T"+str(each["time"]), "| Air Temperature:", str(i["climate"]["air_temp"])+u"\N{DEGREE SIGN}C", "| Surface Temperature:",str(each["surface_temp"])+u"\N{DEGREE SIGN}C", "| Confidence:", each["confidence"])

In [ ]:
# (E) Find the top 10 records with the highest surface temperature (°C).
res_e = list(climate_fire.aggregate([
	{"$sort": {"fire.surface_temp": -1}},
	{"$project": {
		"_id": 0,        
		"date": 1,
		"climate": 1,
		"fire": {
			"$filter": {
				"input": "$fire",
				"as": "fire",
				"cond": {"$eq": ["$$fire.surface_temp", {"$max": "$fire.surface_temp"}]}
			}
		}}
	},
	{"$limit": 10}

]))

pprint(res_e)

In [ ]:
# (F) Find the number of fires each day. Display the date and the total number of
# fires in the output.

res_f = list(climate_fire.aggregate([
	{"$group": {
		"_id": "$date",
		"total_no_fire": { "$sum": { "$size": { '$ifNull': ['$fire', [] ] } } }
		}
	},
	{"$sort": {"_id": pymongo.ASCENDING} }
]))


# Output sorted  by date
for each in res_f:
	print("Date: " + str(each["_id"]) + " | Total no. of Fires: " + str(each["total_no_fire"]))

In [ ]:
# (G) Find the average surface temperature (°C) for each day. Display the date and
# average surface temperature (°C) in the output.

res_g = list(climate_fire.aggregate([
	{"$project": {
		"date": "$date",
		"avg_surface_temp": {"$avg": '$fire.surface_temp'},
		"_id": 0
		}
	}
]))

for each in res_g:
	if each["avg_surface_temp"]:
		each["avg_surface_temp"] = float(round(each["avg_surface_temp"], 2))
	print("Date: " + str(each["date"]) + " | Average Surface Temperature: " + str(each["avg_surface_temp"]))